##part b) Neural network with pytorch
We are going to make neural network with 3 inputs and 2 outputs. Relu function is used for non linearity at each layer. 2 hidden layers are used -one consisting of 5 neurons and second hidden layer consisting of four neurons.We are tryint to predict more than one value. we use dtwo non linear functions-

5x1^2+x2^2+x3^2+5x1+x2+x3
4x1^2+x2^2+2x3^2+4x1+x2+2x3




In [9]:
import numpy as np
import random
from numpy import array
from numpy.random import uniform
from numpy import hstack
import torch
import torch.nn as nn

In [10]:
n=400  # number of input samples
d=3    # number of dimensions
num_hidden1=5 # number of neurons for first hidden layer
num_hidden2=4 # numbe rof neurons for second hidden layer
#x=np.random.uniform(-1,1,(n,d)) # generating data of size=400 X 3
x = torch.FloatTensor(n,d)
w1_true=torch.FloatTensor([[5,1,1],]).T # defining weight matrix for first output neuron
w2_true=torch.FloatTensor([[4,1,1],]).T # defining weight matrix for second output neuron
bias_true=torch.tensor([1])

y1=(x**2) @ w1_true+x @ w1_true+bias_true #final layer output of first neuron 
y2=(x**2) @ w2_true+x @ w2_true+bias_true #final layer output of seconf layer neuron
y_true=hstack((y1, y2)) # joining output of first and second neuron
#lr = .0001
print(f'x: {x.shape},weights:{w1_true.shape}, bias: {bias_true.shape},y:{y_true.shape}')


'''
# Generating data of 3 dim input and 2 dim output
n, d = 500, 3
x = torch.FloatTensor(n, d).uniform_(-1, 1)
weights_true = torch.tensor([[5,1,5],[1,2,1]]).float()
weights_true = torch.transpose(weights_true,0,1)
bias_true = torch.tensor([1,2])
y_true = torch.mm(x**2,weights_true) + torch.mm(x,weights_true) + bias_true
print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')
'''

x: torch.Size([400, 3]),weights:torch.Size([3, 1]), bias: torch.Size([1]),y:(400, 2)


"\n# Generating data of 3 dim input and 2 dim output\nn, d = 500, 3\nx = torch.FloatTensor(n, d).uniform_(-1, 1)\nweights_true = torch.tensor([[5,1,5],[1,2,1]]).float()\nweights_true = torch.transpose(weights_true,0,1)\nbias_true = torch.tensor([1,2])\ny_true = torch.mm(x**2,weights_true) + torch.mm(x,weights_true) + bias_true\nprint(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')\n"

In [11]:
#defining loss function-mse and its derivative  

class MSE:
  def __call__(self, y_pred, y_true):
    self.y_pred = y_pred
    self.y_true = y_true
    return torch.mean((y_pred - y_true) ** 2)

  def backward(self): 
    n = self.y_true.shape[0]
    self.gradient = 2. * (self.y_pred - self.y_true) / n
    return self.gradient

In [12]:
#creating a class to define linear layers , here input dim represents number of input attributes, num_hidden is number of output neurons  
class Linear:
  def __init__(self, input_dim: int, num_hidden: int = 1):
    self.weights = torch.rand(input_dim, num_hidden)
    self.bias = torch.zeros(num_hidden,)
  
  def __call__(self, x):
    self.x = x
    return torch.mm(x, self.weights) + self.bias

  def backward(self, gradient):
    self.weights_gradient = torch.mm(torch.transpose(self.x,0,1), gradient)
    self.bias_gradient = gradient.sum(0)
    self.x_gradient = torch.mm(gradient, torch.transpose(self.weights,0,1))
    return self.x_gradient

  def update(self, lr):
    self.weights = self.weights - lr * self.weights_gradient
    self.bias = self.bias - lr * self.bias_gradient

In [13]:
#defining relu function and its derative
class Relu:
    def __call__(self, input_):
      self.input_ = input_
      self.output = torch.clamp(self.input_, min=0, out=None)
      return self.output
    
    def backward(self, output_gradient):
      self.input_gradient = (self.input_ > 0) * output_gradient 
      return self.input_gradient


In [14]:
#creating Non linear model consisting of 3 layers-2 hidden and one output layer consisting of two output neurons.
from typing import Callable

class Model:
  def __init__(self, input_dim, num_hidden):
    self.linear1 = Linear(input_dim, num_hidden)
    self.relu1 = Relu()
    self.linear2 = Linear(num_hidden, 12) # Pick 12 as number of neurons in hidden layer
    self.relu2 = Relu()
    self.linear3 = Linear(12,2)
  
  # Forward pass
  def __call__(self, x):
    l1 = self.linear1(x)
    r1 = self.relu1(l1)
    l2 = self.linear2(r1)
    r2 = self.relu2(l2)
    l3 = self.linear3(r2)
    return l3
  
  def backward(self, output_gradient):
    linear3_gradient = self.linear3.backward(output_gradient)
    relu2_gradient = self.relu2.backward(linear3_gradient)
    linear2_gradient = self.linear2.backward(relu2_gradient)
    relu1_gradient = self.relu1.backward(linear2_gradient)
    linear1_gradient = self.linear1.backward(relu1_gradient)
    return linear1_gradient

  def update(self, lr):
    self.linear3.update(lr)
    self.linear2.update(lr)
    self.linear1.update(lr)

In [15]:
def fit(x, y, model: Callable, loss: Callable, lr: float, num_epochs: int):
  for epoch in range(num_epochs):
    y_pred = model(x)
    loss_value = loss(y_pred, y)
    if epoch % 50 == 0:
      print(f'Epoch {epoch}, loss {loss_value}')
    gradient_from_loss = loss.backward()
    model.backward(gradient_from_loss)
    model.update(lr)

loss = MSE()
model = Model(d, 20)
fit(x, y_true, model=model, loss=loss, lr=0.0035, num_epochs=1000)



Epoch 0, loss 1.0
Epoch 50, loss 0.5137667655944824
Epoch 100, loss 0.2545018196105957
Epoch 150, loss 0.12607122957706451
Epoch 200, loss 0.062451161444187164
Epoch 250, loss 0.030936097726225853
Epoch 300, loss 0.015324643813073635
Epoch 350, loss 0.007591270375996828
Epoch 400, loss 0.003760454710572958
Epoch 450, loss 0.0018627900863066316
Epoch 500, loss 0.0009227585978806019
Epoch 550, loss 0.00045710214180871844
Epoch 600, loss 0.00022643059492111206
Epoch 650, loss 0.00011216360871912912
Epoch 700, loss 5.5560907640028745e-05
Epoch 750, loss 2.7524100005393848e-05
Epoch 800, loss 1.363439514534548e-05
Epoch 850, loss 6.753877187293256e-06
Epoch 900, loss 3.3453443393227644e-06
Epoch 950, loss 1.6572470258324756e-06


In [16]:
def plot_intereactive_3d(x, y, y_pred=None):
  import plotly.graph_objects as go

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()

In [18]:
from sklearn.manifold import TSNE
X_red = TSNE(n_components=2).fit_transform(x)
y_true_red = TSNE(n_components=1).fit_transform(y_true)
y_pred_red = TSNE(n_components=1).fit_transform(model(x))
print(f'X_red: {X_red.shape}, y_true_red: {y_true_red.shape}, y_pred_red: {y_pred_red.shape}')
plot_intereactive_3d(X_red,y_true_red,y_pred_red)

X_red: (400, 2), y_true_red: (400, 1), y_pred_red: (400, 1)
